### Imports and Preprocessing

In [1]:
# imports
import pandas as pd
import sklearn as skl
import psycopg2
import config as c

# import for multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [3]:
# SQL query
data_sql = """
SELECT *
FROM final_dataset;
"""

# load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)

In [4]:
# get_dummies to encode HallofFameClass column
data_df = pd.get_dummies(data_df, prefix='HOF', columns=['HallOfFameStatus'])
data_df.drop(['HOF_Not Inducted'], axis=1, inplace=True) # remove extra get_dummies column, drop_first no ideal

# drop coulmns/features
data_df.drop(['GUID',
              'Name',
              'HallofFameClass',
              'YearDrafted',
              'TO_YEAR',
              'Years_Played',
              'HOF_Elgibility_Year',
              'Team',
              'College',
              'GP',
              'MIN',
              'idPlayer',
              'BIRTHDATE',
              'POSITION',
              'AGE_ROOKIE_SEASON',
              'Draft_Decade',
              'numBallotsBeforeInduct'], 
             axis=1, inplace=True)

### Optimization Attempt #6: Dropping Draft Features

In [5]:
# dropping draft related features
data_df.drop(['Pick', 'numberRound'], axis=1, inplace=True)
data_df.head()
print(data_df.shape)

,PTS,FGM,FGA,FG%,3P_Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,HOF_Hall of Fame Member
0,6.3,2.8,5.2,53.3,0.0,0.1,0.0,0.8,1.3,62.5,1.7,3.5,5.2,0.4,0.5,0.5,1.0,9.0,0
1,5.6,1.9,6.0,32.3,1.1,3.6,30.1,0.7,0.8,86.1,0.5,1.6,2.0,0.5,0.5,0.3,0.5,4.3,0
2,7.9,2.8,7.0,39.6,0.7,2.4,28.7,1.6,2.1,75.2,1.0,2.2,3.2,1.2,0.7,1.1,1.7,7.6,0
3,5.4,1.9,5.5,35.5,0.6,2.2,26.2,0.9,1.2,75.0,0.6,1.9,2.5,0.9,0.6,0.3,1.0,4.8,0
4,3.7,1.5,3.3,44.9,0.0,0.3,13.3,0.6,1.1,57.4,0.9,2.0,2.9,0.3,0.2,0.4,0.4,4.8,0


(1217, 19)


### Machine Learning

In [6]:
# ML imports
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

In [7]:
# create our features
X = data_df.drop(['HOF_Hall of Fame Member'], axis=1)

# create our target
y = data_df['HOF_Hall of Fame Member']

In [8]:
# normal train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)
Counter(y_train)

# implement ROS resampling
X_resampled, y_resampled = RandomOverSampler(random_state=2, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

# logistic regression using ROS data
model = LogisticRegression(solver='liblinear', random_state=2)
model.fit(X_resampled, y_resampled)

# generate predictions on test set
y_pred_test = model.predict(X_test)

# calculate the accuracy scores
test_acc_score = balanced_accuracy_score(y_test, y_pred_test)
train_acc_score = model.score(X_train, y_train)

# define the confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])

# print results
print('_______DROPPING DRAFT FEATURES_______\n')
print("CONFUSION MATRIX")
display(cm_df)
print("ACCURACY SCORES\n")
print(f'Training Accuracy: {train_acc_score}\n')
print(f'Testing Accuracy: {test_acc_score}\n')
print("CLASSIFICATION REPORT\n")
print(classification_report_imbalanced(y_test, y_pred_test))

Counter({0: 880, 1: 32})

Counter({0: 880, 1: 880})

LogisticRegression(random_state=2, solver='liblinear')

_______DROPPING DRAFT FEATURES_______

CONFUSION MATRIX


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,265,30
HOF Member,2,8


ACCURACY SCORES

Training Accuracy: 0.875

Testing Accuracy: 0.8491525423728814

CLASSIFICATION REPORT

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.90      0.80      0.94      0.85      0.73       295
          1       0.21      0.80      0.90      0.33      0.85      0.71        10

avg / total       0.97      0.90      0.80      0.92      0.85      0.73       305

